In [2]:
import os

In [3]:
from dataclasses import dataclass
from pathlib import Path
CONFIG_FILE_PATH = Path("config\config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")
SCHEMA_FILE_PATH = Path("schema.yaml")

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:

from Mlproject.constants import *
from Mlproject.utils.common import read_yaml, create_directories

In [5]:


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH,
        schema_filepath= SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        if 'artifacts_root' not in self.config:
            self.config['artifacts_root'] = 'artifacts'
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [6]:
import os
import urllib.request as request
import zipfile
from Mlproject import logger
from Mlproject.utils.common import get_size


In [7]:

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info:\n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        


In [16]:

try:
    config = ConfigurationManager()
    print("Config:", config.config)  
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    print(f"An error occurred: {str(e)}")
    raise e

[2024-12-10 14:28:39,285: INFO: common: YAML file: config\config.yaml loaded successfully]
[2024-12-10 14:28:39,669: INFO: common: YAML file: params.yaml loaded successfully]
[2024-12-10 14:28:39,682: INFO: common: YAML file: schema.yaml loaded successfully]
[2024-12-10 14:28:39,721: INFO: common: Created directory at: artifacts]
Config: {'artifact_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/WisdomAnalyst/FIFA-21-DATA-CLEANING-/raw/refs/heads/main/winequality-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'artifacts_root': 'artifacts'}
[2024-12-10 14:28:39,748: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-12-10 14:28:39,788: INFO: 4136036466: File already exists of size: 23 kB]
